# Pollution impact factors for photochemical ozone formation 

Calculate impact factors of photochemical ozone formation. IMPORTANT: If all cells are run you might run out of memory. To avoid this only run the ones you want to calculate e.g. the ones tagged with # NOx for calculating NOx part of the factor. # COMMON cells should always be run.

Exiobase categories:
- NMVOC – combustion – air
- NOx – combustion – air

LC-Impact stressors
- Terrestrial ecosystem damage
    - NOx
    - NMVOC

In [1]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [2]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [ ]:
# no need for aggregation

In [3]:
# COMMON
# diagonalization

# make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

L matrix loaded from pickle


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_16290/1974571270.py:21: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


In [4]:
# NMVOC
diag_nmvoc = exio3_11.satellite.diag_stressor(("NMVOC - combustion - air"))

# calculate S (direct emission multipliers)
if diag_nmvoc.S is None:
    print("Calculating S for NMVOC emissions to air")
    diag_nmvoc.S = pymrio.calc_S(diag_nmvoc.F, exio3_11.x)

diag_nmvoc.D_cba, _, _, _ = pymrio.calc_accounts(diag_nmvoc.S, exio3_11.L, Y_agg)

Calculating S for NMVOC emissions to air


In [5]:
# NOx
diag_nox = exio3_11.satellite.diag_stressor(("NOx - combustion - air"))

# calculate S (direct emission multipliers)
if diag_nox.S is None:
    print("Calculating S for NOx emissions to air")
    diag_nox.S = pymrio.calc_S(diag_nox.F, exio3_11.x)

diag_nox.D_cba, _, _, _ = pymrio.calc_accounts(diag_nox.S, exio3_11.L, Y_agg)

Calculating S for NOx emissions to air


## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [6]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns_nox = {}
for series_name, series in diag_nox.D_cba.items():
    series_sum = series.sum()
    columns_nox[series_name] = series / series_sum

dr_s_nox = pd.DataFrame(columns_nox)
dr_s_nox

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.344358e-07   
       Cereal grains nec                                  6.374609e-07   
       Vegetables, fruit, nuts                            1.265168e-08   
       Oil seeds                                          3.107563e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       3.589327e-08   
       Recreational, cultural and sporting services (92)  4.511124e-07   
       Other services (93)                                8.180205e-07   
       Private households with employed persons (95)      1.181847e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.618657e-01   
       Cereal grains nec                                  8.248140e-06   
       Vegetables, fruit, nuts                            6.380320e-07   
       Oil seeds                                          9.136562e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       5.383362e-08   
       Recreational, cultural and sporting services (92)  8.064633e-07   
       Other services (93)                                9.947005e-07   
       Private households with employed persons (95)      2.178709e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.314786e-05   
       Cereal grains nec                                      5.237391e-01   
       Vegetables, fruit, nuts                                1.367913e-06   
       Oil seeds                                              1.655661e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           6.504425e-08   
       Recreational, cultural and sporting services (92)      1.109607e-06   
       Other services (93)                                    1.235349e-06   
       Private households with employed persons (95)          2.825305e-08   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        3.668645e-05   
       Cereal grains nec                                            5.581509e-05   
       Vegetables, fruit, nuts                                      1.980606e-01   
       Oil seeds                                                    3.727199e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 9.428543e-07   
       Recreational, cultural and sporting services (92)     

In [7]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns_nmvoc = {}
for series_name, series in diag_nmvoc.D_cba.items():
    series_sum = series.sum()
    columns_nmvoc[series_name] = series / series_sum

dr_s_nmvoc = pd.DataFrame(columns_nmvoc)
dr_s_nmvoc

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.589616e-07   
       Cereal grains nec                                  1.009805e-06   
       Vegetables, fruit, nuts                            1.718345e-08   
       Oil seeds                                          4.913447e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.401985e-07   
       Recreational, cultural and sporting services (92)  4.452550e-07   
       Other services (93)                                4.543608e-07   
       Private households with employed persons (95)      1.274698e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              3.160335e-01   
       Cereal grains nec                                  1.427522e-05   
       Vegetables, fruit, nuts                            9.467770e-07   
       Oil seeds                                          1.578309e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       2.297349e-07   
       Recreational, cultural and sporting services (92)  8.696643e-07   
       Other services (93)                                6.036316e-07   
       Private households with employed persons (95)      2.567370e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.053256e-05   
       Cereal grains nec                                      6.016811e-01   
       Vegetables, fruit, nuts                                1.347374e-06   
       Oil seeds                                              1.898477e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           1.842497e-07   
       Recreational, cultural and sporting services (92)      7.942567e-07   
       Other services (93)                                    4.976154e-07   
       Private households with employed persons (95)          2.209932e-08   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        3.578127e-05   
       Cereal grains nec                                            7.806817e-05   
       Vegetables, fruit, nuts                                      2.375193e-01   
       Oil seeds                                                    5.203402e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 3.251723e-06   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [8]:
# COMMON
# harmonize regions 

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    # custom mappings for countries that pycountry does not recognize
    # these should cover all the countries in the LCI data if country has alpha-2 code
    # these mappings were extracted manually
    extra_mappings = {
        "Turkey": "TR",
        "Russia": "RU",
        "Bahamas, The": "BS",
        "Bonaire": "BQ",
        "Byelarus": "BY",
        "Brunei": "BN",
        "Cape Verde": "CV",
        "Cocos Islands": "CC",
        "Congo DRC": "CD",
        "China, Hong Kong Special Administrative Region": "HK",
        "Curacao": "CW",
        "Democratic Republic of the Congo": "CD",
        "Falkland Islands": "FK",
        "Falkland Islands (Islas Malvinas)": "FK",
        "Gambia, The": "GM",
        "Gaza Strip": "PS",
        "Heard Island & McDonald Islands": "HM",
        "Ivory Coast": "CI",
        "Macedonia": "MK",
        "The Former Yugoslav Republic of Macedonia": "MK",
        "Macau": "MO",
        "Man, Isle of": "IM",
        "Micronesia": "FM",
        "Myanmar (Burma)": "MM",
        "Netherlands Antilles": "AN",
        "Palestinian Territory": "PS",
        "Pacific Islands (Palau)": "PW",
        "Pitcairn Islands": "PN",
        "Reunion": "RE",
        "Saba": "BQ",
        "Saint Eustatius": "BQ",
        "Saint Helena": "SH",
        "Saint Martin": "MF",
        "Sint Maarten": "SX",
        "South Georgia and the South Sandwich Is": "GS",
        "South Georgia": "GS",
        "St. Helena": "SH",
        "Saint Barthelemy": "BL",
        "Saint Kitts and Nevis": "KN",
        "St. Kitts and Nevis": "KN",
        "St. Lucia": "LC",
        "St. Pierre and Miquelon": "PM",
        "Sao Tomo and Principe": "ST",
        "St. Vincent and the Grenadines": "VC",
        "Svalbard": "SJ",
        "Jan Mayen": "SJ",
        "Swaziland": "SZ",
        "US Virgin Islands": "VI",
        "Virgin Islands": "VG",
        "Western Samoa": "WS",
        "West Bank": "PS",
    }
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Alpha-2 country code does not exist for ", name)
            return None

In [9]:
# COMMON
# load and prepare lc-impact data
lci = pd.read_excel(arguments["lc_impact_path"] + "/5-photochemical ozone formation/Photochemical_Ozone_formation.xlsx", sheet_name="CFs per country", skiprows=0, header=[0,1])
lci.dropna(inplace=True)

# Flatten the multi-level columns and rename them
lci.columns = [' '.join(col).strip() for col in lci.columns]
lci.rename(columns={lci.columns[0]: "Country", lci.columns[4]: "NOx", lci.columns[5]: "NMVOC"}, inplace=True)
lci.drop(columns=[lci.columns[1], lci.columns[2], lci.columns[3]], inplace=True)

# Add country codes to the LCI data and drop rows without country codes
lci["Country_Code"] = lci["Country"].apply(get_country_code)
lci.dropna(subset=["Country_Code"], inplace=True)
lci

Alpha-2 country code does not exist for  Russia Europe


,Country,NOx,NMVOC,Country_Code
0,Afghanistan,2.900000e-15,1.100000e-16,AF
1,Albania,2.100000e-15,4.900000e-16,AL
2,Algeria,4.900000e-15,7.700000e-16,DZ
3,Angola,1.900000e-16,2.600000e-17,AO
4,Argentina,9.500000e-16,-1.800000e-17,AR
...,...,...,...,...
184,Vietnam,1.300000e-15,1.300000e-16,VN
185,Western Sahara,4.900000e-15,7.700000e-16,EH
186,Yemen,1.600000e-15,2.000000e-16,YE
187,Zambia,1.900000e-16,2.600000e-17,ZM


In [ ]:
# COMMON
# add regional averages for regions that are not in LCI data

def get_missing_from_lci(exio_regions, lci):
    """
    Get the regions that are in exiobase but not in lci data.
    """
    missing = []
    for region in exio_regions:
        if region not in lci["Country_Code"].tolist():
            missing.append(region)
    return missing

exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
if len(get_missing_from_lci(exio_regions_without_row, lci)) > 0:
    # No augmentation needed for ozone
    print("Missing from LCI:", get_missing_from_lci(exio_regions_without_row, lci))
    assert len(get_missing_from_lci(exio_regions_without_row, lci)) == 0, "There are still missing regions after augmentation"

In [11]:
# COMMON
# harmonize regions in LCI data

def get_row_regions(lci_country_codes, exio_country_codes):
    """
    Get the country codes from lci countries that don't exist in exiobase i.e. rest of the world countries.
    """
    row_regions = []
    for country in lci_country_codes:
        if country not in exio_country_codes:
            row_regions.append(country)
    
    # find duplicates in the list
    duplicates = []
    unique_regions = []
    seen_once = set()
    for item in row_regions:
        if item not in seen_once:
            unique_regions.append(item)
            seen_once.add(item)
        else:
            duplicates.append(item)
    if duplicates:
        print("Duplicates found in row regions:", duplicates)
    return unique_regions

row_countries = get_row_regions(lci["Country_Code"].tolist(), exio_regions)
print("Row regions:", row_countries)

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

Row regions: ['AF', 'AL', 'DZ', 'AO', 'AR', 'AM', 'AW', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BZ', 'BJ', 'BT', 'BO', 'BA', 'BW', 'BN', 'BF', 'BI', 'BY', 'KH', 'CM', 'CV', 'CF', 'TD', 'CL', 'HK', 'CO', 'KM', 'CG', 'CR', 'CU', 'CD', 'DJ', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'ET', 'FJ', 'GF', 'GA', 'GM', 'GE', 'GH', 'GL', 'GD', 'GP', 'GT', 'GN', 'GW', 'GY', 'HT', 'HN', 'IS', 'IR', 'IQ', 'IL', 'CI', 'JM', 'JO', 'KZ', 'KE', 'KW', 'KG', 'LA', 'LB', 'LS', 'LR', 'LY', 'MK', 'MG', 'MW', 'MY', 'MV', 'ML', 'MQ', 'MR', 'MU', 'MD', 'MN', 'MA', 'MZ', 'MM', 'NA', 'NP', 'AN', 'NZ', 'NI', 'NE', 'NG', 'KP', 'OM', 'PK', 'PA', 'PG', 'PY', 'PE', 'PH', 'PR', 'QA', 'RE', 'RW', 'LC', 'VC', 'WS', 'SA', 'SN', 'RS', 'SL', 'SG', 'SB', 'SO', 'LK', 'SD', 'SR', 'SZ', 'SY', 'ST', 'TJ', 'TZ', 'TH', 'TG', 'TO', 'TT', 'TN', 'TM', 'UG', 'UA', 'AE', 'UY', 'UZ', 'VU', 'VE', 'VN', 'EH', 'YE', 'ZM', 'ZW']


In [18]:
# NOx
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl_nox = dr_s_nox.loc["WL"].copy()
wl_nox = wl_nox / len(row_american_countries)

we_nox = dr_s_nox.loc["WE"].copy()
we_nox = we_nox / len(row_eu_countries)

wa_nox = dr_s_nox.loc["WA"].copy()
wa_nox = wa_nox / len(row_asia_pacific_countries)

wf_nox = dr_s_nox.loc["WF"].copy()
wf_nox = wf_nox / len(row_african_countries)

wm_nox = dr_s_nox.loc["WM"].copy()
wm_nox = wm_nox / len(row_middle_eastern_countries)

dr_u_nox = dr_s_nox.copy()
dr_u_nox = dr_u_nox.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region_nox = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region_nox[region] = we_nox
    elif region in row_asia_pacific_countries:
        country_to_region_nox[region] = wa_nox
    elif region in row_african_countries:
        country_to_region_nox[region] = wf_nox
    elif region in row_american_countries:
        country_to_region_nox[region] = wl_nox
    elif region in row_middle_eastern_countries:
        country_to_region_nox[region] = wm_nox
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region_nox[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx_nox = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data_nox = pd.concat(all_data)
combined_data_nox.index = combined_idx_nox

dr_u_nox = pd.concat([dr_u_nox, combined_data_nox])
# drop row region columns
dr_u_nox = dr_u_nox.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u_nox

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.344358e-07   
   Cereal grains nec                                  6.374609e-07   
   Vegetables, fruit, nuts                            1.265168e-08   
   Oil seeds                                          3.107563e-08   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       2.380440e-10   
   Recreational, cultural and sporting services (92)  9.172334e-10   
   Other services (93)                                1.152041e-09   
   Private households with employed persons (95)      4.360964e-11   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.618657e-01   
   Cereal grains nec                                  8.248140e-06   
   Vegetables, fruit, nuts                            6.380320e-07   
   Oil seeds                                          9.136562e-07   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       1.008397e-09   
   Recreational, cultural and sporting services (92)  5.392866e-09   
   Other services (93)                                3.827194e-09   
   Private households with employed persons (95)      1.814742e-10   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.314786e-05   
   Cereal grains nec                                      5.237391e-01   
   Vegetables, fruit, nuts                                1.367913e-06   
   Oil seeds                                              1.655661e-06   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           1.250269e-09   
   Recreational, cultural and sporting services (92)      7.665677e-09   
   Other services (93)                                    4.769292e-09   
   Private households with employed persons (95)          1.866963e-10   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        3.668645e-05   
   Cereal grains nec                                            5.581509e-05   
   Vegetables, fruit, nuts                                      1.980606e-01   
   Oil seeds                                                    3.727199e-06   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 1.053325e-08   
   Recreational, cultural and sporting services (92)            1.427826e-08   
   Other services (93)                                          1.168259e-08   
   Private households with employed persons (95)                4.420830e-10   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

In [16]:
# NMVOC
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s_nmvoc.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s_nmvoc.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s_nmvoc.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s_nmvoc.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s_nmvoc.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u_nmvoc = dr_s_nmvoc.copy()
dr_u_nmvoc = dr_u_nmvoc.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_nmvoc = pd.concat([dr_u_nmvoc, combined_data])
# drop row region columns
dr_u_nmvoc = dr_u_nmvoc.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u_nmvoc

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.589616e-07   
   Cereal grains nec                                  1.009805e-06   
   Vegetables, fruit, nuts                            1.718345e-08   
   Oil seeds                                          4.913447e-08   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       2.124262e-09   
   Recreational, cultural and sporting services (92)  6.148190e-09   
   Other services (93)                                6.071508e-09   
   Private households with employed persons (95)      1.628378e-09   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              3.160335e-01   
   Cereal grains nec                                  1.427522e-05   
   Vegetables, fruit, nuts                            9.467770e-07   
   Oil seeds                                          1.578309e-06   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       9.831623e-09   
   Recreational, cultural and sporting services (92)  3.949391e-08   
   Other services (93)                                2.203699e-08   
   Private households with employed persons (95)      7.403392e-09   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.053256e-05   
   Cereal grains nec                                      6.016811e-01   
   Vegetables, fruit, nuts                                1.347374e-06   
   Oil seeds                                              1.898477e-06   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           8.091365e-09   
   Recreational, cultural and sporting services (92)      3.726366e-08   
   Other services (93)                                    1.822847e-08   
   Private households with employed persons (95)          5.055640e-09   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        3.578127e-05   
   Cereal grains nec                                            7.806817e-05   
   Vegetables, fruit, nuts                                      2.375193e-01   
   Oil seeds                                                    5.203402e-06   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 8.299497e-08   
   Recreational, cultural and sporting services (92)            8.450479e-08   
   Other services (93)                                          5.436341e-08   
   Private households with employed persons (95)                1.457522e-08   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [20]:
# NOx

# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_nox = dr_u_nox.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total_nox = satellite_cleaned.loc["NOx - combustion - air"]
scalars = total_nox.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_nox.shape) 

# multiply each column of dr_u by the respective column value from exio3_19 impact factors
dr_f_nox = dr_f_nox * scalars # same as dr_f * diag(scalars) but more efficient with numpy broadcasting
dr_f_nox

8800
(37400, 8800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  1.426843e+03   
   Cereal grains nec                                        0.0  4.494214e-02   
   Vegetables, fruit, nuts                                  0.0  3.476483e-03   
   Oil seeds                                                0.0  4.978293e-03   
...                                                         ...           ...   
ZW Membership organisation services n.e.c. (91)             0.0  5.494511e-06   
   Recreational, cultural and sporting services (92)        0.0  2.938443e-05   
   Other services (93)                                      0.0  2.085346e-05   
   Private households with employed persons (95)            0.0  9.888094e-07   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  3.892483e-02   
   Cereal grains nec                                      1.550554e+03   
   Vegetables, fruit, nuts                                4.049768e-03   
   Oil seeds                                              4.901661e-03   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           3.701479e-06   
   Recreational, cultural and sporting services (92)      2.269459e-05   
   Other services (93)                                    1.411971e-05   
   Private households with employed persons (95)          5.527231e-07   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        2.522068e-02   
   Cereal grains nec                                            3.837096e-02   
   Vegetables, fruit, nuts                                      1.361599e+02   
   Oil seeds                                                    2.562322e-03   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 7.241249e-06   
   Recreational, cultural and sporting services (92)            9.815815e-06   
   Other services (93)                                          8.031383e-06   
   Private households with employed persons (95)                3.039170e-07   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                 \
                                                      Oil seeds   
AT Paddy rice                                          0.000000   
   Wheat                                               0.011826   
   Cereal grains nec                                   0.012776   
   Vegetables, fruit, nuts                             0.001133   
   Oil seeds                                          26.279675   
...                                                         ...   
ZW Membership organisation services n.e.c. (91)        0.000006   
   Recreational, cultural and sporting services (92)   0.000022   
   Other services (93)                                 0.000020   
   Private households with employed persons (95)       0.000001   
   Extra-territorial organizations and bodies          0.000000   

                                                                             \
                                                    

In [21]:
# NMVOC

# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_nmvoc = dr_u_nmvoc.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total_nmvoc = satellite_cleaned.loc["NMVOC - combustion - air"]
scalars = total_nmvoc.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_nmvoc.shape) 

# multiply each column of dr_u by the respective column value from exio3_19 impact factors
dr_f_nmvoc = dr_f_nmvoc * scalars
dr_f_nmvoc

8800
(37400, 8800)


AT              \
                                                     Paddy rice       Wheat   
AT Paddy rice                                               0.0    0.000000   
   Wheat                                                    0.0  325.462539   
   Cereal grains nec                                        0.0    0.014701   
   Vegetables, fruit, nuts                                  0.0    0.000975   
   Oil seeds                                                0.0    0.001625   
...                                                         ...         ...   
ZW Membership organisation services n.e.c. (91)             0.0    0.000010   
   Recreational, cultural and sporting services (92)        0.0    0.000041   
   Other services (93)                                      0.0    0.000023   
   Private households with employed persons (95)            0.0    0.000008   
   Extra-territorial organizations and bodies               0.0    0.000000   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                 0.000000   
   Wheat                                                      0.007794   
   Cereal grains nec                                        445.262195   
   Vegetables, fruit, nuts                                    0.000997   
   Oil seeds                                                  0.001405   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)               0.000006   
   Recreational, cultural and sporting services (92)          0.000028   
   Other services (93)                                        0.000013   
   Private households with employed persons (95)              0.000004   
   Extra-territorial organizations and bodies                 0.000000   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                       0.000000   
   Wheat                                                            0.004482   
   Cereal grains nec                                                0.009778   
   Vegetables, fruit, nuts                                         29.749670   
   Oil seeds                                                        0.000652   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                     0.000010   
   Recreational, cultural and sporting services (92)                0.000011   
   Other services (93)                                              0.000007   
   Private households with employed persons (95)                    0.000002   
   Extra-territorial organizations and bodies                       0.000000   

                                                                \
                                                     Oil seeds   
AT Paddy rice                                         0.000000   
   Wheat                                              0.002676   
   Cereal grains nec                                  0.004145   
   Vegetables, fruit, nuts                            0.000315   
   Oil seeds                                          8.510953   
...                                                        ...   
ZW Membership organisation services n.e.c. (91)       0.000010   
   Recreational, cultural and sporting services (92)  0.000030   
   Other services (93)                                0.000022   
   Private households with employed persons (95)      0.000010   
   Extra-territorial organizations and bodies         0.000000   

                                                                             \
                                                     Sugar cane, sugar beet   
AT Paddy ric

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [23]:
# NMVOC
lci_copy_nmvoc = lci.copy()
lci_copy_nmvoc.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy_nmvoc = lci_copy_nmvoc[~lci_copy_nmvoc.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_w
lci_reindexed_nmvoc = lci_copy_nmvoc.reindex(dr_f_nmvoc.index.get_level_values(0).unique())

cf_nmvoc = lci_reindexed_nmvoc["NMVOC"].to_numpy()
cf_nmvoc = np.repeat(cf_nmvoc, 200)
cf_nmvoc = np.tile(cf_nmvoc, (dr_f_nmvoc.shape[1], 1)).T

pdf_nmvoc = dr_f_nmvoc * cf_nmvoc
pdf_total_nmvoc = pdf_nmvoc.sum()

In [24]:
# NOx
lci_copy_nox = lci.copy()
lci_copy_nox.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy_nox = lci_copy_nox[~lci_copy_nox.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_nox
lci_reindexed_nox = lci_copy_nox.reindex(dr_f_nox.index.get_level_values(0).unique())

cf_nox = lci_reindexed_nox["NOx"].to_numpy()
cf_nox = np.repeat(cf_nox, 200)
cf_nox = np.tile(cf_nox, (dr_f_nox.shape[1], 1)).T

pdf_nox = dr_f_nox * cf_nox
pdf_total_nox = pdf_nox.sum()

In [25]:
# NMVOC
pd.DataFrame(pdf_total_nmvoc).to_csv("csv/pdf-pollution-ozone-nmvoc.csv", index=True)

In [26]:
# NOx
pd.DataFrame(pdf_total_nox).to_csv("csv/pdf-pollution-ozone-nox.csv", index=True)